# Sign Game Server

Game Plan:

In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import models
from tensorflow.keras import Sequential, layers
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

2023-03-07 16:57:51.841766: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-07 16:57:52.273197: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-07 16:57:52.273227: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-07 16:57:53.769565: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

## Later Code

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense
# from tensorflow.keras.callbacks import EarlyStopping


# def initialize_model():
#     model = Sequential()
#     model.add(LSTM(units=, activation='tanh', return_sequence=True, inputshape=()))
#     model.add(LSTM(units=, activation='tanh', return_sequences=True))
#     model.add(LSTM(units=, activation='tanh', return_sequences=False))
#     model.add(Dense(, activation='relu'))
#     model.add(Dense(, activation='relu'))
#     model.add(Dense(26, activation='softmax'))
#     return model

# def compile_model(model):
#     model.compile(loss='categorical_crossentropy',
#                   optimizer='adam',
#                   metrics=['accuracy'])
#     return model

# def train_model(model: Model,
#                 X: np.ndarray,
#                 y: np.ndarray):
#     es = EarlyStopping(monitor='val_loss',
#                        patience=,
#                        restore_best_weights=True,
#                        verbose=0)
    
#     history = model.fit(X, y, epochs=1000, callbacks=[es], verbose=0)
#     return model, history

## Dataset Cleaning

In [3]:
# Dataset
df_train = pd.read_csv('../sample_sets/sample_training.csv')

In [4]:
df_train.shape

(1033, 64)

In [5]:
df_train.columns

Index(['class_type', 'wristX', 'wristY', 'wristZ', 'thumb_CmcX', 'thumb_CmcY',
       'thumb_CmcZ', 'thumb_McpX', 'thumb_McpY', 'thumb_McpZ', 'thumb_IpX',
       'thumb_IpY', 'thumb_IpZ', 'thumb_TipX', 'thumb_TipY', 'thumb_TipZ',
       'index_McpX', 'index_McpY', 'index_McpZ', 'index_PipX', 'index_PipY',
       'index_PipZ', 'index_DipX', 'index_DipY', 'index_DipZ', 'index_TipX',
       'index_TipY', 'index_TipZ', 'middle_McpX', 'middle_McpY', 'middle_McpZ',
       'middle_PipX', 'middle_PipY', 'middle_PipZ', 'middle_DipX',
       'middle_DipY', 'middle_DipZ', 'middle_TipX', 'middle_TipY',
       'middle_TipZ', 'ring_McpX', 'ring_McpY', 'ring_McpZ', 'ring_PipX',
       'ring_PipY', 'ring_PipZ', 'ring_DipX', 'ring_DipY', 'ring_DipZ',
       'ring_TipX', 'ring_TipY', 'ring_TipZ', 'pinky_McpX', 'pinky_McpY',
       'pinky_McpZ', 'pinky_PipX', 'pinky_PipY', 'pinky_PipZ', 'pinky_DipX',
       'pinky_DipY', 'pinky_DipZ', 'pinky_TipX', 'pinky_TipY', 'pinky_TipZ'],
      dtype='object')

In [7]:
X_train = df_train.drop(columns = ['class_type'])

In [8]:
df_train["class_type"] = pd.Categorical(df_train["class_type"])
df_train["class_type"] = df_train.class_type.cat.codes

In [10]:
y_train=df_train['class_type']
y_train=to_categorical(y_train, num_classes=26)

In [11]:
X_train.shape, y_train.shape

((1033, 63), (1033, 26))

In [13]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [20]:
X_train = np.reshape(np.array(X_train), (X_train.shape[0], X_train.shape[1], 1))
X_train.shape

(1033, 63, 1)

## CNN Model

In [21]:
# Architecture
model = models.Sequential()
model.add(layers.Conv1D(filters=32, kernel_size=5, strides=1, padding='causal', input_shape=X_train.shape[1:3], activation='relu'))
model.add(layers.Conv1D(filters=32, kernel_size=5, strides=1, padding='causal', activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(filters=64, kernel_size=5, strides=1, padding='causal', activation='relu'))
model.add(layers.Conv1D(filters=64, kernel_size=5, strides=1, padding='causal', activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(filters=128, kernel_size=5, strides=1, padding='causal', activation='relu'))
model.add(layers.Conv1D(filters=128, kernel_size=5, strides=1, padding='causal', activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(filters=256, kernel_size=5, strides=1, padding='causal', activation='relu'))
model.add(layers.Conv1D(filters=256, kernel_size=5, strides=1, padding='causal', activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Dropout(0.2))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(26, activation='softmax'))

In [22]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 63, 32)            192       
                                                                 
 conv1d_2 (Conv1D)           (None, 63, 32)            5152      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 31, 32)           0         
 )                                                               
                                                                 
 conv1d_3 (Conv1D)           (None, 31, 64)            10304     
                                                                 
 conv1d_4 (Conv1D)           (None, 31, 64)            20544     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 15, 64)           0         
 1D)                                                  

In [25]:
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
33/33 [==============================] - 1s 22ms/step - loss: 2.1716 - accuracy: 0.2924
Epoch 2/100
33/33 [==============================] - 1s 22ms/step - loss: 2.0696 - accuracy: 0.3136
Epoch 3/100
33/33 [==============================] - 1s 22ms/step - loss: 1.7348 - accuracy: 0.4134
Epoch 4/100
33/33 [==============================] - 1s 24ms/step - loss: 1.7214 - accuracy: 0.4153
Epoch 5/100
33/33 [==============================] - 1s 24ms/step - loss: 1.6081 - accuracy: 0.4569
Epoch 6/100
33/33 [==============================] - 1s 23ms/step - loss: 1.4837 - accuracy: 0.4898
Epoch 7/100
33/33 [==============================] - 1s 22ms/step - loss: 1.3242 - accuracy: 0.5431
Epoch 8/100
33/33 [==============================] - 1s 25ms/step - loss: 1.2713 - accuracy: 0.5818
Epoch 9/100
33/33 [==============================] - 1s 22ms/step - loss: 1.2591 - accuracy: 0.5857
Epoch 10/100
33/33 [==============================] - 1s 22ms/step - loss: 1.1005 - accuracy: 0.6244

33/33 [==============================] - 1s 22ms/step - loss: 0.2102 - accuracy: 0.9371
Epoch 83/100
33/33 [==============================] - 1s 23ms/step - loss: 0.2266 - accuracy: 0.9187
Epoch 84/100
33/33 [==============================] - 1s 23ms/step - loss: 0.1473 - accuracy: 0.9468
Epoch 85/100
33/33 [==============================] - 1s 23ms/step - loss: 0.1077 - accuracy: 0.9632
Epoch 86/100
33/33 [==============================] - 1s 22ms/step - loss: 0.1147 - accuracy: 0.9603
Epoch 87/100
33/33 [==============================] - 1s 25ms/step - loss: 0.0949 - accuracy: 0.9642
Epoch 88/100
33/33 [==============================] - 1s 23ms/step - loss: 0.1145 - accuracy: 0.9555
Epoch 89/100
33/33 [==============================] - 1s 23ms/step - loss: 0.1289 - accuracy: 0.9574
Epoch 90/100
33/33 [==============================] - 1s 25ms/step - loss: 0.1097 - accuracy: 0.9613
Epoch 91/100
33/33 [==============================] - 1s 23ms/step - loss: 0.1592 - accuracy: 0.9497
Epo

# Validation Set

In [27]:
df_val = pd.read_csv('../sample_sets/sample_validation.csv')

In [28]:
X_val = df_val.drop(columns = ['class_type'])
y_val = df_val['class_type']

In [31]:
df_val["class_type"] = pd.Categorical(df_val["class_type"])
df_val["class_type"] = df_val.class_type.cat.codes

In [32]:
y_val=df_val['class_type']
y_val=to_categorical(y_val, num_classes=26)

In [33]:
X_val = np.reshape(np.array(X_val), (X_val.shape[0], X_val.shape[1], 1))
X_val.shape

(204, 63, 1)

In [34]:
model.evaluate(X_val, y_val)

7/7 [==============================] - 0s 7ms/step - loss: 1.3824 - accuracy: 0.8235


[1.3824421167373657, 0.8235294222831726]